# Data extraction, transformation, and loading into a JSON file
This is part of the project described in <https://github.com/amchagas/OSH_papers_DB>, check the project readme for more details.

This notebook loads data sources and merges them in a single compressed JSON file.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import unicodedata
import string
import rispy
import matplotlib.pyplot as plt
from pathlib import Path
from project_definitions import baseDir, dataSourceDir, dataOutDir, figDir, articleDataFile
from project_definitions import store_data, load_data
from pprint import pprint
import html
from urllib.parse import unquote
from jellyfish import damerau_levenshtein_distance as edit_distance

## Sources

In [ ]:
scieloSource = {
    'paths': [dataSourceDir / x for x in ("scielo.ris",)],
    'rispy_args': {},
    'col_rename': {},
    'transforms': [],
}
scopusSource = {
    'paths': [dataSourceDir / x for x in ("scopus.ris",)],
    'rispy_args': {},
    'col_rename': {},
    'transforms': [],
}
wosSource = {
    'paths': [
        dataSourceDir / x for x in "wos1001-1500.ciw  wos1-500.ciw  wos1501-1689.ciw  wos501-1000.ciw".split()
    ],
    'rispy_args': {'implementation': 'wok'},
    'col_rename': {'publication_year': 'year', 'document_title': 'title'},
    'transforms': [],
}

In [ ]:
def load_source(dataSource):
    dfs = []
    for path in dataSource['paths']:
        with path.open() as f:
            df = pd.DataFrame(rispy.load(f, **dataSource['rispy_args']))
        df['__source'] = [[path.name] for _ in range(len(df))]
        dfs.append(df)
    cdf = pd.concat(dfs, join='outer', ignore_index=True)
    cdf = cdf.rename(columns=dataSource['col_rename'])
    for trans in dataSource['transforms']:
        cdf = cdf.transform(trans)
    return cdf.sort_index(axis=1)

In [ ]:
scieloData = load_source(scieloSource)

In [ ]:
scopusData = load_source(scopusSource)

In [ ]:
wosData = load_source(wosSource)

In [ ]:
allDataList = [scieloData, scopusData, wosData]

In [ ]:
allData = pd.concat(allDataList, join='outer', ignore_index=True)

In [ ]:
# Keep only article data
article_data = allData.loc[allData["type_of_reference"].eq('JOUR') | allData["publication_type"].eq('J')]

In [ ]:
# Normalize DOI
article_data.loc[:, 'doi'] = article_data['doi'].str.translate(
    str.maketrans(string.ascii_lowercase, string.ascii_uppercase)
)

In [ ]:
# Remove spurious records
article_data = article_data.loc[article_data['url'].ne(
    "https://www.scopus.com/inward/record.uri?eid=2-s2.0-85052219975&partnerID=40&md5=7b54756675a6d510c9db069b49b634d6"
)]

In [ ]:
# Correct faulty records
data_corrections = {
    'doi': {
        r'^(.*)/PDF$': r'\1',
    }
}
corrected_article_data = article_data.replace(data_corrections, regex=True)
article_data.compare(corrected_article_data)

In [ ]:
article_data = corrected_article_data

In [ ]:
article_data.describe()

In [ ]:
def merge_series_keep_longest(sx):
    if sx.isna().all():
        return np.nan
    if sx.name == '__source':
        return sx.sum()
    if sx.name == 'doi':
        if len(sx.dropna().unique()) > 1:
            print('Warning, merging different DOIs:\n', sx)
            return list(sx.dropna().unique())
    return sx[sx.map(len, na_action='ignore').idxmax()] # Keep a list of all DOIs - must explode before using!

def merge_records_keep_longest(dfx):
    return dfx.agg(merge_series_keep_longest)

In [ ]:
# Merge data with same DOI
article_doi = article_data.groupby(article_data['doi'].values).agg(merge_records_keep_longest)

# Reassemble data with and without DOI
article_nodoi = article_data[~article_data.doi.isin(article_doi.index)]
article_data = pd.concat([article_doi, article_nodoi], ignore_index=True)

In [ ]:
def remove_diacritics(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
def clean_titles(sx):
    return (
        sx
        .str.lower()
        .str.replace(r'[^\s\w]', ' ', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
        # .map(remove_diacritics) # no need as our corpus is in English
    )

In [ ]:
class Match:
    """
    Index string values with similar strings under the same index, for use in a `groupby`.

    First normalizes titles. Then, for each value, returns the index of the first previously indexed value
    whose edit_distance is <= threshold, or a new index if none is found.
    """
    def __init__(self, df, threshold=0):
        self.df = df
        assert not df['title'].hasnans
        self.titles = clean_titles(self.df['title'])
        self.threshold = threshold
        self.match_index = {}
    def match(self, x):
        x = self.titles.loc[x]
        if x in self.match_index:
            return self.match_index[x]
        if self.threshold > 0:
            for m, idx in self.match_index.items():
                if edit_distance(x, m) <= self.threshold:
                    self.match_index[x] = idx
                    return self.match_index[x]
        self.match_index[x] = len(self.match_index)
        return self.match_index[x]

In [ ]:
articles_g = article_data.groupby(Match(article_data, 5).match)

In [ ]:
aa = articles_g.agg(list)[articles_g.size() > 1]

In [ ]:
# Test alternatives matchers
if False:
    articles_gx = article_data.groupby(Match(article_data, 15).match)
    bb = articles_gx.agg(list)[articles_gx.size() > 1]
    pprint([sorted(x) for x in (
        set(clean_titles(aa.explode('title')['title'])).difference(clean_titles(bb.explode('title')['title'])),
        set(clean_titles(bb.explode('title')['title'])).difference(clean_titles(aa.explode('title')['title'])),
    )])

In [ ]:
def clean_name(name):
    return remove_diacritics(name.split(',')[0].split(' ')[-1].lower().replace(' ', '').replace('-', ''))

In [ ]:
# Check that matching titles also have matching year
sel = aa['year'].map(lambda x: len(set(x)) > 1)
aa[sel]

In [ ]:
# Check that matching titles also have matching author (impl: first author last name)
sel = aa['authors'].map(
    lambda merged_authors: set(
        tuple( # last name of each author
            clean_name(author)
            for author in authors
        )
        for authors in merged_authors
        if not ( isinstance(authors, float) and pd.isna(authors) ) # skip NANs
    )
).map(
    lambda merged_lastnames: sum(
        edit_distance(firstauthor, other_firstauthor) # sum the edit distances
        for merged_firstauthor in list(zip(*merged_lastnames))[:1] # first authors
        for i, firstauthor in enumerate(merged_firstauthor)
        for other_firstauthor in merged_firstauthor[i+1:] # distinct pairs
    )
) > 0
aa[sel].authors.to_dict()

In [ ]:
article_data[['doi', 'title', 'authors']].describe()

In [ ]:
article_data = articles_g.agg(merge_records_keep_longest)
article_data

In [ ]:
# Store deduplicated data and check the stored version reproduces the data
store_data(article_data, articleDataFile)
assert article_data.equals(load_data(articleDataFile))

# Load article data (if already stored from the code above)

In [ ]:
article_data = load_data(articleDataFile)

## PLOS Collection sources

In [ ]:
plosData = pd.read_csv('https://raw.githubusercontent.com/amchagas/open-source-toolkit/main/plos-items.csv')

In [ ]:
sel_article = plosData[
    "Content Type (URL items only - Research Article, Web Article, Commentary, Video, Poster)"
].eq("Research Article")
sel_hardware = plosData["Hardware or software"].eq("hardware")
plosData = plosData.loc[sel_article & sel_hardware]

### DOIs

In [ ]:
assert plosData["URI (DOI or URL)"].notna().all()
# Normalize DOI
plosData["URI (DOI or URL)"] = plosData["URI (DOI or URL)"].str.translate(
    str.maketrans(string.ascii_lowercase, string.ascii_uppercase)
)

In [ ]:
# Get the doi and doi-like, fixing doi-like containing extra stuff
re_doi = r"(10\.[1-9]\d{3,}(?:\.\d+)*/.+)"
re_http_doi_fix = r"HTTPS?://.*/" + re_doi + r"(?:/|/FULL|/ABSTRACT|#\w+)$"

In [ ]:
plosData_doi = plosData['URI (DOI or URL)'].str.extract(re_doi)[0]

In [ ]:
plosData_doi_http_doi_fixed = (
    plosData['URI (DOI or URL)']
    .str.extract(re_http_doi_fix)[0]
    .map(unquote, na_action='ignore')
)

In [ ]:
plosData_doi.loc[plosData_doi_http_doi_fixed.notna()].compare(plosData_doi_http_doi_fixed.dropna())

In [ ]:
assert 'doi' not in plosData
plosData['doi'] = plosData_doi_http_doi_fixed.where(plosData_doi_http_doi_fixed.notna(), plosData_doi)

In [ ]:
plosData['doi'].dropna()

In [ ]:
print(
    len(set(plosData['doi'].dropna()).intersection(article_data['doi'].explode())),
    len(set(plosData['doi'].dropna()).symmetric_difference(article_data['doi'].explode())),
)

### Titles

In [ ]:
plosData['Title (URL items only)'] = plosData['Title (URL items only)'].str.strip()

In [ ]:
# How many from the collection have their title in article_data
plosData['Title (URL items only)'].pipe(clean_titles).map(
    lambda x: article_data['title'].pipe(clean_titles).str.contains(rf'(?i){x}', regex=True).any()
).sum()

In [ ]:
# How many from the collection have their title in article_data if we require they have DOIs
plosData['Title (URL items only)'].loc[plosData['doi'].notna()].pipe(clean_titles).map(
    lambda x: article_data.loc[article_data['doi'].notna()].title.pipe(clean_titles).str.contains(rf'(?i){x}', regex=True).any()
).sum()

In [ ]:
# Give me 10 from the collection having DOIs
z = plosData['doi'].dropna().sample(10)
print(z)

In [ ]:
# Get their titles if their titles are not in article_data
for i, title in plosData.loc[z.index]['Title (URL items only)'].pipe(clean_titles).items():
    if not clean_titles(article_data['title']).str.contains(rf'(?i){title}', regex=True).any():
        print(i, title)

In [ ]:
# Selector for DOIs only in the collection
sel_new_doi = ~plosData["doi"].dropna().isin(article_data['doi'].explode().values)
sel_new_doi.sum()

In [ ]:
# Selector for Titles only in the collection
sel_new_title = ~clean_titles(plosData["Title (URL items only)"]).isin(clean_titles(article_data['title']))
sel_new_title.sum()

In [ ]:
# Same title, different DOIs
x = plosData[["doi", "Title (URL items only)"]].loc[sel_new_doi & ~sel_new_title]
for i, y in x["Title (URL items only)"].str.lower().items():
    print(
        y,
        article_data["doi"].loc[
            article_data['title'].str.lower().eq(y)
        ].squeeze(),
        plosData.loc[i, 'doi']
    )
    

In [ ]:
# Same DOI, different titles
x = plosData.loc[~sel_new_doi & sel_new_title, 'doi']
for y in x:
    print(
        plosData.loc[plosData['doi'].eq(y), "Title (URL items only)"].squeeze(),
        article_data.loc[article_data['doi'].explode().eq(y), 'title'].squeeze(),
    )

# All done, now just mess around

In [ ]:
article_data.shape

In [ ]:
article_data.issn.str.replace('[^\d]', '', regex=True).value_counts()

In [ ]:
article_data.issn.str.replace('[^\d]', '', regex=True).value_counts().reset_index().plot(loglog=True)

In [ ]:
article_data.groupby('year').size().plot.bar()

## Play with our 10 article sample

In [ ]:
dois = pd.Series("""
    10.1371/journal.pone.0187219
    10.1371/journal.pone.0059840
    10.1371/journal.pone.0030837
    10.1371/journal.pone.0118545
    10.1371/journal.pone.0206678
    10.1371/journal.pone.0143547
    10.1371/journal.pone.0220751
    10.1371/journal.pone.0107216
    10.1371/journal.pone.0226761
    10.1371/journal.pone.0193744
""".split()).str.translate(
    str.maketrans(string.ascii_lowercase, string.ascii_uppercase)
)

In [ ]:
dois[dois.isin(article_data.doi.explode())]